In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

# Allow importing from parent directory by temporarily moving the CWD up one level
# Very hacky, but there literally isn't a simpler way (in Jupyter)
import sys
sys.path.append("..")
from common import get_dataframe_from_pipeline
outages = get_dataframe_from_pipeline("../pipeline/3.csv.gz")
# Drop the path back down after import
sys.path.pop()

### Checking if the distances are normally distributed:

In [ ]:
stats.normaltest(outages['outageToSubstationDistance']).pvalue

### First Idea
1. Splitting the distances into **equal** length **bins** and seeing how many outages occured that have distances in that distance interval for example from (0,3), is the number of outages that had a distance of between 0 to 3 to the nearest station.
2. Now we **groupby** these intervals and aggregate by counting the number of outages that are in this interval. This gives us outages per distance
3. We can do a statistical test like a **T-test** to see if there is a significant different between the first half of these outages compared to the second half of these outages.

In [ ]:
#this cuts the data into 3 equal width bins.
data = pd.Series([2,19,1,20, 13, 19, 24, 30])
bins = pd.cut(data, bins=3)
print(bins)

In [ ]:
n = 10 #number of bins
distance_bins = pd.cut(outages['outageToSubstationDistance'], bins=n)
outages['distance_bin'] = distance_bins
outages_per_dist = outages.groupby(['distance_bin']).size()
outages_per_dist = pd.DataFrame(outages_per_dist)
outages_per_dist = outages_per_dist.rename(columns={0:"#of outages"})
outages_per_dist


### Checking for correlations: **distance vs timeout**
Checking if there is any correlation between the distance of the outage to the station and the time it took for the outage to be resolved.

In [ ]:
outages['timeOut'] = outages['dateOn'] - outages['dateOff']
outages['timeOut'] = outages['timeOut'].apply(lambda x: x.total_seconds()/60)
outages['timeOut']
#timeout is the total of minutes without power (we can change it into hours if its better)

In [ ]:
stats.normaltest(outages['timeOut']).pvalue

In [ ]:
fit = stats.linregress(outages['timeOut'], outages['outageToSubstationDistance'])
plt.xticks(rotation = 25)
plt.plot(outages["timeOut"], outages["outageToSubstationDistance"], 'b.', alpha = 0.5)
plt.plot(outages["timeOut"], outages["timeOut"]*fit.slope + fit.intercept, 'r-', linewidth = 3)
plt.show()
#this plot doesnt look good because the distance csv that I created was not very good. 
#it would be nice to try it on our actual big dataset and the corresponding distances csv

In [ ]:
outages["timeOut"].corr(outages["outageToSubstationDistance"])

### Trying transformations:

In [ ]:
outages["timeOut"].apply(np.sqrt).corr(outages["outageToSubstationDistance"].apply(np.sqrt))

### Log transformation:


In [ ]:
outages["timeOut"].apply(np.log).corr(outages["outageToSubstationDistance"].apply(np.log))

In [ ]:
timeouts_transformed = outages["timeOut"].apply(np.log)
distance_transformed = outages["outageToSubstationDistance"].apply(np.log)
fit = stats.linregress(timeouts_transformed, distance_transformed)
plt.xticks(rotation = 25)
plt.plot(timeouts_transformed, distance_transformed, 'b.', alpha = 0.5)
plt.plot(timeouts_transformed, timeouts_transformed*fit.slope + fit.intercept, 'r-', linewidth = 3)
plt.show()

Using log makes better bins as well! 
but are the bins even meaningful?

In [ ]:
n = 10 #number of bins
distance_bins = pd.cut(outages['outageToSubstationDistance'].apply(np.log), bins=n)
outages['distance_bin'] = distance_bins
outages_per_dist = outages.groupby(['distance_bin']).size()
outages_per_dist = pd.DataFrame(outages_per_dist)
outages_per_dist = outages_per_dist.rename(columns={0:"#of outages"})
outages_per_dist